# ETL Processes

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=pestcidedb user=bangzhu")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

## Explore pestcide_data and dictionary

In [4]:
pestcide_files = [f for f in get_files('data') if not f.endswith('dictionary.csv')]

In [5]:
filepath = pestcide_files[0]
print(filepath)

/Users/bangzhu/study/data/capstone/pestcide/data/2015.csv


In [6]:
df = pd.read_csv(filepath)
df.head()

,compound_name,year,state_code,county_code,low_est,high_est
0,1-Methyl Cyclopropene,2015,41,1,NaN,0.0
1,1-Methyl Cyclopropene,2015,41,21,NaN,0.1
2,1-Methyl Cyclopropene,2015,41,27,NaN,1.7
3,1-Methyl Cyclopropene,2015,41,49,NaN,0.1
4,1-Methyl Cyclopropene,2015,41,55,NaN,0.0


In [7]:
county_dictionary_file = os.getcwd() + '/data/dictionary.json'
print(county_dictionary_file)

/Users/bangzhu/study/data/capstone/pestcide/data/dictionary.json


In [8]:
county_df = pd.read_json(county_dictionary_file)
county_df.head()

,state_code,county_code,county_name,state_name
0,1,1,Autauga County,AL
1,1,3,Baldwin County,AL
2,1,5,Barbour County,AL
3,1,7,Bibb County,AL
4,1,9,Blount County,AL


## Process pestcide_data

### #1 `compounds` Table

In [12]:
compound_data = df[df['compound_name']=='2,4-D'][['compound_name']].values[0].tolist()
compound_data

['2,4-D']

In [13]:
compound_data = compound_data*2
cur.execute(compound_table_insert, compound_data)
conn.commit()

### #2 `state` Table

In [14]:
state_data = county_df[['state_code', 'state_name']].values[0].tolist()
state_data

[1, 'AL']

In [15]:
cur.execute(state_table_insert, state_data)
conn.commit()

### #3 `county` Table

In [16]:
county_data = county_df[['state_code', 'county_code', 'county_name']].values[0].tolist()
county_data

[1, 1, 'Autauga County']

In [17]:
county_data = county_data + [county_data[0], county_data[1]]
cur.execute(county_table_insert, county_data)
conn.commit()

### #4 `pestcide_usage` Table

In [18]:
pestcide_data = df.loc[(df['state_code']==1) & (df['county_code']==1) & (df['compound_name']=='2,4-D')].values[0].tolist()
pestcide_data

['2,4-D', 2015, 1, 1, 3243.5, 3363.2]

In [19]:
cur.execute('select compound_id from compounds where compound_name=%s', [pestcide_data[0]])
compound_id = cur.fetchone()[0]
compound_id

2

In [20]:
cur.execute('select county_id from counties where state_code=%s and county_code=%s', [pestcide_data[2], pestcide_data[3]])
county_id = cur.fetchone()[0]
county_id

1

In [21]:
pestcide_usage_data = (compound_id, pestcide_data[1], county_id, pestcide_data[4], pestcide_data[5])
cur.execute(pestcide_usage_table_insert, pestcide_usage_data)
conn.commit()

# Test data

In [22]:
cur.execute('select * from pestcide_use limit 5')
cur.fetchone()

(1, 2, 2015, 1, Decimal('3243.5'), Decimal('3363.2'))

# Close Connection

In [23]:
conn.close()